In [2]:
import pickle, gzip
import numpy as np

# Load the dataset
f = gzip.open('C:\\Users\\kshitij\\Desktop\\coursera asg\\datasets\\mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f,encoding='latin1')
f.close()

In [3]:
x, y = train_set
print(x.shape)

(50000, 784)


In [39]:
xtest, ytest = test_set
Xtest = xtest.T
ytest = ytest.reshape(1,10000)
n = Xtest.shape[1]
Ytest = np.zeros((10,n))
for i in range(n):
    Ytest[int(ytest[0,i]),i]=1
print(ytest)
print(Ytest)

[[7 2 1 ... 4 5 6]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
X = x.T

In [5]:
y=y.reshape(1,50000)

In [6]:
print(X.shape)

(784, 50000)


In [7]:
m=X.shape[1]


In [8]:
Y = np.zeros((10,m))
for i in range(m):
    Y[int(y[0,i]),i]=1

In [9]:
def initialize_parameters(dims):
    parameters={}
    l = len(dims)
    for i in range(1,l):
        parameters['W'+str(i)] = np.random.randn(dims[i],dims[i-1])*0.01
        parameters['b'+str(i)] = np.zeros((dims[i],1))
    return parameters

In [10]:
def relu(Z):
    x = np.maximum(0,Z)
    return x

In [11]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x/np.sum(e_x, axis=0, keepdims=True)

In [12]:
def forward_propagation(X, parameters):
    cache = {}
    l = len(parameters)//2
    cache['A0'] = X
    for i in range(1,l+1):
        cache['Z'+str(i)] = np.dot(parameters['W'+str(i)],cache['A'+str(i-1)]) + parameters['b'+str(i)]
        
        cache['A'+str(i)] = relu(cache['Z'+str(i)])
        
    cache['A'+str(l)] = softmax(cache['Z'+str(l)])
            
    return cache

In [13]:
def compute_cost(AL,Y):
    m = Y.shape[1]
    cost = -1*np.sum(Y*np.log(AL+0.0001))/m
    return cost

In [14]:
def backRelu(Z):
    c = (Z + np.abs(Z))/(2*np.abs(Z))
    return c

In [15]:
def back_propagation(X, Y, parameters, cache):
    m = Y.shape[1]
    l = len(parameters)//2
    back_cache = {}
    
    dZ = cache['A'+str(l)] - Y
    back_cache['dW'+str(l)] = (1/m)*np.dot(dZ, cache['A'+str(l-1)].T)
    back_cache['db'+str(l)] = (1/m)*np.sum(dZ, axis=1, keepdims = True)
    
    for i in reversed(range(1,l)):
        dZ = np.dot(parameters['W'+str(i+1)].T,dZ)*backRelu(cache['Z'+str(i)])
        back_cache['dW'+str(i)] = (1/m)*np.dot(dZ, cache['A'+str(i-1)].T)
        back_cache['db'+str(i)] = (1/m)*np.sum(dZ, axis=1, keepdims=True)
     
    return back_cache

In [16]:
def update(paramters, back_cache, lr):
    l = len(parameters)//2
    for i in range(1,l+1):
        parameters['W'+str(i)] -= lr*back_cache['dW'+str(i)]
        parameters['b'+str(i)] -= lr*back_cache['db'+str(i)]
        
    return parameters
        

In [76]:
parameters = initialize_parameters([784, 500, 100, 10])

In [77]:
k=0
for i in range(10):
    k=0
    for j in range(500):
        cache = forward_propagation(X[:,k:k+100], parameters)
        c = compute_cost(cache['A'+str(len([784, 500,100, 10])-1)],Y[:,k:k+100])
        
        grads = back_propagation(X, Y[:,k:k+100], parameters, cache)

        parameters = update(parameters, grads, 0.3)
        k+=100
        #print('Z3 =',cache['Z3'])
    #print("A3=",cache['A3'])
    print(c)   
    


0.18546273991904122
0.10834453335203417
0.0762114730060142
0.04392859834626282
0.02932663598462521
0.028175665150923956
0.019035915518081755
0.017119801907605058
0.007647222624127607
0.022080914468939603


In [57]:
A = cache['A3'].T


In [58]:

temp = []
for row in A:
    temp.append(np.argmax(row))
print(temp)    

[6, 8, 1, 0, 4, 5, 7, 2, 2, 7, 6, 2, 3, 3, 2, 6, 2, 2, 7, 1, 4, 7, 5, 7, 0, 9, 3, 1, 5, 2, 6, 6, 1, 7, 4, 5, 2, 4, 3, 0, 4, 1, 4, 1, 9, 9, 3, 8, 3, 0, 5, 2, 0, 0, 0, 3, 9, 5, 7, 6, 6, 1, 2, 8, 1, 6, 3, 4, 3, 6, 6, 0, 1, 9, 0, 2, 2, 6, 7, 4, 7, 3, 4, 9, 9, 3, 7, 0, 9, 0, 8, 5, 5, 2, 4, 5, 0, 8, 4, 8]


In [59]:
cv = y[:,-100:]
count=0
for i in range(len(temp)):
    if temp[i]==cv[0][i]:
        count+=1
print(cv)

[[6 8 1 0 9 5 7 2 2 7 6 2 3 3 2 6 2 2 7 1 4 7 5 7 0 9 3 1 5 2 6 6 1 7 4 5
  2 4 3 0 4 1 4 1 9 9 3 8 3 0 5 2 0 0 0 3 9 5 7 6 6 1 2 8 1 6 3 4 3 6 6 0
  1 9 0 2 2 6 7 4 7 3 4 9 9 3 7 0 9 0 8 5 5 2 4 5 0 8 4 8]]


In [60]:
print(count/100)

0.99


In [97]:
import pickle
f=open('C:\\Users\\kshitij\\Desktop\\coursera asg\\datasets\\miniparameters.pckl','wb')
pickle.dump(parameters,f)
f.close()

In [61]:

cache = forward_propagation(Xtest[:,0:100], parameters)
c = compute_cost(cache['A'+str(len([784, 500,100, 10])-1)],Ytest[:,0:100])
c

0.06519127048346732

In [62]:

fg = ytest[0]
gf = fg[0:100]
Ah = cache['A3'].T

temp = []
for row in Ah:
    temp.append(np.argmax(row))
    
count = 0
for i in range(len(temp)):
    if temp[i]==gf[i]:
        count+=1

print(count/100)

0.98


In [78]:
xtest.shape

(10000, 784)

In [92]:
lt = 8999
t = xtest[lt,0:784]
t = t.reshape(784,1)

In [94]:
print(ytest)
ans = ytest[0][lt]
print(ans)
ansm = np.zeros((10,1))
ansm[0,0]=1
ansm
print(t.shape)

[[7 2 1 ... 4 5 6]]
0
(784, 1)


In [95]:

cache = forward_propagation(t, parameters)
c = compute_cost(cache['A'+str(len([784, 500,100, 10])-1)],ansm)
A = cache['A3'].T
A

array([[9.99999907e-01, 4.05433282e-15, 6.25249382e-08, 1.85462735e-13,
        4.92705648e-14, 6.96076617e-10, 2.73325805e-08, 3.42842055e-10,
        1.01880060e-11, 1.82420431e-09]])

In [96]:
print(np.argmax(A))

0
